In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False

In [2]:
import sys
import os

In [3]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

False

In [8]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")): 
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..
        
    %pip install -qr requirements.txt  --quiet
    setup_already_done = True

/home/silverio/Projects/uni/DeepLearning/VideoPrediction_MovingMNIST
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
# from torch import optim, nn, utils, Tensor
# import torchvision
# from torchvision.transforms import ToTensor
import lightning.pytorch as pl

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [5]:
# !nvidia-smi

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")

# Visualization Test
dataset.visualize(0)


In [10]:
loader = DataLoader(dataset)

In [17]:
from src.simpleLSTM import simpleLSTM

model = simpleLSTM()


In [18]:
for data in loader:
    print(model.training_step(data,0))
    break

tensor(2893.5317, grad_fn=<MseLossBackward0>)


/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


In [19]:
# from lightning.pytorch.loggers.wandb import WandbLogger
# logger = WandbLogger()

In [21]:
# trainer = pl.Trainer(limit_train_batches=1, max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, accelerator="gpu")
trainer = pl.Trainer(max_epochs=5, limit_train_batches=10)

trainer.fit(model=model, train_dataloaders=loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | relu    | ReLU       | 0     
2 | encoder | Sequential | 4.2 M 
3 | lstm    | LSTM       | 3.1 M 
4 | decoder | Sequential | 4.7 M 
---------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.275    Total estimated model params size (MB)
/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
# !zip -r ../logs lightning_logs/ 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/ #--port 6007

In [ ]:
# import gc
# print(torch.cuda.list_gpu_processes())
# gc.collect()
# torch.cuda.empty_cache()